# Neural PDE Option Greeks — Milestone Report Run

This notebook reproduces the end-to-end experiment described in the project proposal: generate the full synthetic dataset, train the PINN with boundary and PDE losses, and evaluate price/Greek accuracy plus diagnostic surfaces.


## 1. Overview
This research notebook mirrors the experimental protocol described in the accompanying report. Each stage is self-contained so results can be reproduced or stress-tested under alternative settings.

1. **Environment** – establish paths and execution device.
2. **Configuration** – declare dataset and optimisation hyperparameters.
3. **Data Preparation** – synthesise Black–Scholes samples for all splits.
4. **Model Training** – fit the PINN with adaptive sampling and gradient regularisation.
5. **Validation Diagnostics** – benchmark against analytic Greeks in-distribution.
6. **Out-of-Sample Benchmark** – evaluate generalisation vs. baseline estimators.


In [ ]:
# 0) Environment
import os
import sys
from pathlib import Path
import numpy as np
import torch

import plotly.io as pio

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / 'src').exists():
    for parent in PROJECT_ROOT.parents:
        if (parent / 'src').exists():
            PROJECT_ROOT = parent
            break
    else:
        raise RuntimeError("Could not locate project root containing 'src'.")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
DATA_DIR = PROJECT_ROOT / 'data'
RESULTS_DIR = PROJECT_ROOT / 'results'
FIGURES_DIR = PROJECT_ROOT / 'figures'
RUN_TAG = 'milestone_report_full_run'
RUN_RESULTS_DIR = RESULTS_DIR / RUN_TAG
FIG_DIR = FIGURES_DIR / RUN_TAG
RUN_RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print('Using device:', device)

pio.renderers.default = 'notebook'


## 2. Experimental Configuration
Hyperparameters are grouped into `CONFIG['data']` and `CONFIG['train']` for clarity. The defaults reproduce the settings from our core experiment; edit in-place to explore ablations or stress scenarios.


In [ ]:
# 1) Configuration
from pprint import pprint

SUCCESS_TARGETS = {
    'price_rmse': 1e-2,
    'delta_mae': 5e-2,
    'gamma_mae': 1e-1,
    'theta_mae': 5e-2,
    'vega_mae': 5e-2,
    'rho_mae': 5e-2,
    'gamma_tv_ratio': 2.0,
    'ood_price_rmse_ratio': 1.10,
    'ood_delta_mae_ratio': 1.10,
    'ood_gamma_mae_ratio': 1.10,
    'ood_theta_mae_ratio': 1.10,
    'ood_vega_mae_ratio': 1.10,
    'ood_rho_mae_ratio': 1.10,
}

FULL_RUN_CONFIG = {
    'run_name': RUN_TAG,
    'contract': {
        'K': 100.0,
        'T': 2.0,
        'r': 0.05,
        't_min': 0.01,
        't_max': 1.999,
        's_bounds': (20.0, 200.0),
        'sigma_bounds': (0.05, 0.6),
    },
    'data': {
        'n_train': 1_000_000,
        'n_val':   100_000,
        'n_test':  100_000,
        'seed':    123,
    },
    'train': {
        'epochs': 100,
        'lr': 5e-4,
        'warmup_base_lr': 1e-5,
        'batch_size': 2048,
        'adaptive_sampling': True,
        'adaptive_every': 5,
        'adaptive_points': 20_000,
        'adaptive_radius': 0.1,
        'adaptive_eval_samples': 75_000,
        'use_warmup': True,
        'warmup_steps': 1_000,
        'grad_clip': 1.0,
        'lambda_reg': 0.01,
        'boundary_weight': 1.0,
        'boundary_warmup': 25,
    },
    'evaluation': {
        'val_subset': 50_000,
        'sample_size': 50_000,
        'mc_paths': 100_000,
        'surface_grid': 60,
        'gamma_tv_grid': 75,
        'ood_sigma': (0.60, 0.65),
        'ood_samples': 50_000,
    },
    'success_targets': SUCCESS_TARGETS,
}

# Quick validation-oriented configuration
CONFIG = {
    'run_name': RUN_TAG,
    'contract': {
        'K': 100.0,
        'T': 2.0,
        'r': 0.05,
        't_min': 0.01,
        't_max': 1.999,
        's_bounds': (20.0, 200.0),
        'sigma_bounds': (0.05, 0.6),
    },
    'data': {
        'n_train': 200_000,
        'n_val':   40_000,
        'n_test':  40_000,
        'seed':    123,
    },
    'train': {
        'epochs': 50,
        'lr': 5e-4,
        'warmup_base_lr': 1e-5,
        'batch_size': 4_096,
        'adaptive_sampling': True,
        'adaptive_every': 5,
        'adaptive_points': 10_000,
        'adaptive_radius': 0.08,
        'adaptive_eval_samples': 50_000,
        'use_warmup': True,
        'warmup_steps': 500,
        'grad_clip': 1.0,
        'lambda_reg': 0.01,
        'boundary_weight': 0.3,
        'boundary_warmup': 25,
    },
    'evaluation': {
        'val_subset': 40_000,
        'sample_size': 40_000,
        'mc_paths': 50_000,
        'surface_grid': 30,
        'gamma_tv_grid': 50,
        'ood_sigma': (0.60, 0.65),
        'ood_samples': 20_000,
    },
    'success_targets': SUCCESS_TARGETS,
}


pprint(CONFIG)


## 3. Data Preparation
We draw $(S,t,\sigma)$ uniformly over the ranges specified in the configuration and label each sample with the Black–Scholes closed-form price. Time is bounded away from maturity to avoid numerical instability. The generated arrays are persisted to `data/` for auditability.


In [ ]:
# 2) Data generation (train/val/test)
from src.data import generate_dataset
contract = CONFIG['contract']
splits = generate_dataset(
    n_train=CONFIG['data']['n_train'],
    n_val=CONFIG['data']['n_val'],
    n_test=CONFIG['data']['n_test'],
    seed=CONFIG['data']['seed'],
    output_dir=DATA_DIR,
    K=contract['K'],
    T=contract['T'],
    r=contract['r'],
    t_min=contract['t_min'],
    t_max=contract['t_max'],
    s_bounds=contract['s_bounds'],
    sigma_bounds=contract['sigma_bounds'],
)
list(splits.keys())

## 4. Model Training
The training routine applies Adam with warmup, gradient clipping, and adaptive sampling concentrated on high-PDE-residual regions. The gradient penalty weight `lambda_reg` moderates surface smoothness, especially for Δ and Γ.


In [ ]:
# 3) Train PINN
from src.train import train

checkpoint_path = RUN_RESULTS_DIR / 'pinn_checkpoint.pt'
plot_path = FIG_DIR / 'loss_curves.html'
log_path = RUN_RESULTS_DIR / 'training_history.json'

train_cfg = CONFIG['train']
model, history = train(
    num_workers=os.cpu_count(),
    pin_memory=(device.type == 'cuda'),
    epochs=train_cfg['epochs'],
    lr=train_cfg['lr'],
    warmup_base_lr=train_cfg['warmup_base_lr'],
    batch_size=train_cfg['batch_size'],
    data_path=DATA_DIR / 'synthetic_train.npy',
    val_path=DATA_DIR / 'synthetic_val.npy',
    checkpoint_path=checkpoint_path,
    device=device,
    adaptive_sampling=train_cfg['adaptive_sampling'],
    adaptive_every=train_cfg['adaptive_every'],
    adaptive_points=train_cfg['adaptive_points'],
    adaptive_radius=train_cfg['adaptive_radius'],
    adaptive_eval_samples=train_cfg['adaptive_eval_samples'],
    use_warmup=train_cfg['use_warmup'],
    warmup_steps=train_cfg['warmup_steps'],
    grad_clip=train_cfg['grad_clip'],
    lambda_reg=train_cfg['lambda_reg'],
    boundary_weight=train_cfg['boundary_weight'],
    boundary_warmup=train_cfg['boundary_warmup'],
    plot_losses=True,
    plot_path=plot_path,
    log_path=log_path,
)
len(history)


### 4.1 Loss Trajectories
The Plotly dashboards below provide both linear-scale and log-scale views of the loss components so we can verify balance between the data-fit term and the physics residual.


In [ ]:
# 4) Review saved loss curves
from IPython.display import display, HTML
loss_html = FIG_DIR / 'loss_curves.html'
loss_log_html = FIG_DIR / 'loss_curves_log.html'
labels = [
    (loss_html, 'Linear scale'),
    (loss_log_html, 'Log scale (L_price, L_PDE, L_reg, L_boundary)'),
]
for html_path, label in labels:
    if html_path.exists():
        display(HTML(f"<h4>{label}</h4>" + html_path.read_text()))
    else:
        print(f"{html_path.name} not found")


## 5. Validation Diagnostics & Smoothness Checks
We evaluate the trained PINN on a validation subset to compute pricing RMSE and MAE for each Greek
against the analytic Black--Scholes references. In parallel we probe smoothness by measuring the
total variation of the predicted Gamma surface over a mid-maturity $(S, \sigma)$ grid, mirroring the
smoothness criterion in the milestone report.

In [ ]:
# 5) Validation metrics & smoothness checks
from numpy.random import default_rng
from src.preprocessing import normalize_inputs, load_normalization_config
from src.utils.black_scholes import bs_price, bs_greeks

model.eval()
rng = default_rng(CONFIG['data']['seed'])
norm_config = load_normalization_config(DATA_DIR)
val = np.load(DATA_DIR / 'synthetic_val.npy')

subset_n = min(CONFIG['evaluation']['val_subset'], len(val))
if subset_n < len(val):
    idx = rng.choice(len(val), size=subset_n, replace=False)
    val_sample = val[idx]
else:
    val_sample = val

S_np, t_np, sigma_np, price_np = val_sample.T
contract = CONFIG['contract']

def pinn_forward_np(S_values, t_values, sigma_values):
    S_tensor = torch.tensor(S_values, dtype=torch.float32, device=device, requires_grad=True)
    t_tensor = torch.tensor(t_values, dtype=torch.float32, device=device, requires_grad=True)
    sigma_tensor = torch.tensor(sigma_values, dtype=torch.float32, device=device, requires_grad=True)

    features = normalize_inputs(S_tensor, t_tensor, sigma_tensor, config=norm_config)
    features = features.detach().clone().requires_grad_(True)
    pred = model(features).squeeze()

    ones = torch.ones_like(pred)
    grad_feats = torch.autograd.grad(
        pred,
        features,
        grad_outputs=ones,
        create_graph=True,
        retain_graph=True,
    )[0]

    grad_x_norm = grad_feats[:, 0]
    grad_tau_norm = grad_feats[:, 1]
    grad_sigma_norm = grad_feats[:, 2]

    d2_feats = torch.autograd.grad(
        grad_x_norm,
        features,
        grad_outputs=torch.ones_like(grad_x_norm),
        create_graph=True,
        retain_graph=True,
    )[0]
    d2V_dx_norm2 = d2_feats[:, 0]

    x_range = max(norm_config.x_max - norm_config.x_min, 1e-6)
    tau_range = max(norm_config.tau_range, 1e-6)
    sigma_range = max(norm_config.sigma_range, 1e-6)

    dx_norm_dx = 2.0 / x_range
    dtau_norm_dtau = 2.0 / tau_range
    dsigma_norm_dsigma = 2.0 / sigma_range

    dV_dx = grad_x_norm * dx_norm_dx
    delta = dV_dx / S_tensor

    d2V_dx2 = d2V_dx_norm2 * (dx_norm_dx**2)
    gamma = d2V_dx2 * (1.0 / (S_tensor**2)) + dV_dx * (-1.0 / (S_tensor**2))

    theta = -(grad_tau_norm * dtau_norm_dtau)
    vega = grad_sigma_norm * dsigma_norm_dsigma

    outputs = dict(
        price=pred.detach().cpu().numpy(),
        delta=delta.detach().cpu().numpy(),
        gamma=gamma.detach().cpu().numpy(),
        theta=theta.detach().cpu().numpy(),
        vega=vega.detach().cpu().numpy(),
    )
    return outputs

val_outputs = pinn_forward_np(S_np, t_np, sigma_np)
analytic = bs_greeks(S_np, norm_config.K, norm_config.T, t_np, sigma_np, norm_config.r)
tau_np = np.clip(norm_config.T - t_np, 1e-6, None)
rho_pred = tau_np * (S_np * val_outputs['delta'] - val_outputs['price'])

val_metrics = {
    'price_rmse': float(np.sqrt(np.mean((val_outputs['price'] - price_np) ** 2))),
    'delta_mae': float(np.mean(np.abs(val_outputs['delta'] - analytic['delta']))),
    'gamma_mae': float(np.mean(np.abs(val_outputs['gamma'] - analytic['gamma']))),
    'theta_mae': float(np.mean(np.abs(val_outputs['theta'] - analytic['theta']))),
    'vega_mae': float(np.mean(np.abs(val_outputs['vega'] - analytic['vega']))),
    'rho_mae': float(np.mean(np.abs(rho_pred - analytic['rho']))),
}

# Gamma surface total variation on mid-maturity slice
grid_n = CONFIG['evaluation']['gamma_tv_grid']
S_grid = np.linspace(contract['s_bounds'][0], contract['s_bounds'][1], grid_n)
sigma_grid = np.linspace(contract['sigma_bounds'][0], contract['sigma_bounds'][1], grid_n)
S_mesh, Sigma_mesh = np.meshgrid(S_grid, sigma_grid, indexing='ij')
t_slice = np.full_like(S_mesh, 0.5 * (contract['t_min'] + contract['t_max']))

gamma_grid_pred = pinn_forward_np(
    S_mesh.reshape(-1),
    t_slice.reshape(-1),
    Sigma_mesh.reshape(-1),
)['gamma'].reshape(S_mesh.shape)
gamma_grid_true = bs_greeks(
    S_mesh,
    norm_config.K,
    norm_config.T,
    t_slice,
    Sigma_mesh,
    norm_config.r,
)['gamma']

def total_variation(surface: np.ndarray) -> float:
    return float(np.abs(np.diff(surface, axis=0)).sum() + np.abs(np.diff(surface, axis=1)).sum())

val_metrics['gamma_tv_pred'] = total_variation(gamma_grid_pred)
val_metrics['gamma_tv_true'] = total_variation(gamma_grid_true)
val_metrics['gamma_tv_ratio'] = val_metrics['gamma_tv_pred'] / max(val_metrics['gamma_tv_true'], 1e-12)

val_metrics


## 6. Volatility Stress Test $ (\sigma \in [0.60, 0.65]) $
To quantify out-of-distribution robustness we evaluate the model on options drawn from a higher
volatility band than the training data. Metrics are reported alongside their degradation relative to
the validation split, matching the OOD criterion in the milestone report.

In [ ]:
# 6) Volatility stress metrics
sigma_low, sigma_high = CONFIG['evaluation']['ood_sigma']
ood_samples = CONFIG['evaluation']['ood_samples']
ood_rng = default_rng(CONFIG['data']['seed'] + 7)

S_ood = ood_rng.uniform(contract['s_bounds'][0], contract['s_bounds'][1], size=ood_samples)
t_ood = ood_rng.uniform(contract['t_min'], contract['t_max'], size=ood_samples)
sigma_ood = ood_rng.uniform(sigma_low, sigma_high, size=ood_samples)

price_ood = bs_price(S_ood, norm_config.K, norm_config.T, t_ood, sigma_ood, norm_config.r)
ood_outputs = pinn_forward_np(S_ood, t_ood, sigma_ood)
analytic_ood = bs_greeks(S_ood, norm_config.K, norm_config.T, t_ood, sigma_ood, norm_config.r)
tau_ood = np.clip(norm_config.T - t_ood, 1e-6, None)
rho_ood_pred = tau_ood * (S_ood * ood_outputs['delta'] - ood_outputs['price'])

ood_metrics = {
    'price_rmse': float(np.sqrt(np.mean((ood_outputs['price'] - price_ood) ** 2))),
    'delta_mae': float(np.mean(np.abs(ood_outputs['delta'] - analytic_ood['delta']))),
    'gamma_mae': float(np.mean(np.abs(ood_outputs['gamma'] - analytic_ood['gamma']))),
    'theta_mae': float(np.mean(np.abs(ood_outputs['theta'] - analytic_ood['theta']))),
    'vega_mae': float(np.mean(np.abs(ood_outputs['vega'] - analytic_ood['vega']))),
    'rho_mae': float(np.mean(np.abs(rho_ood_pred - analytic_ood['rho']))),
}

ood_ratios = {
    key: ood_metrics[key] / val_metrics[key] if val_metrics.get(key) else np.nan
    for key in ('price_rmse', 'delta_mae', 'gamma_mae', 'theta_mae', 'vega_mae', 'rho_mae')
}

ood_metrics['ratios_vs_val'] = ood_ratios
ood_metrics


## 7. Out-of-Sample Benchmark
We invoke the evaluation utility to benchmark the PINN against finite-difference and Monte Carlo baselines on held-out data. The function logs metrics and exports interactive plots (2D overlays, error histograms, and 3D surfaces for price, Δ, Γ, Θ, ν, and ρ).

In [ ]:
# 7) Out-of-sample evaluation & visuals
from src.test import evaluate_oos
import json

oos_metrics = evaluate_oos(
    data_path=DATA_DIR / 'synthetic_test.npy',
    model_path=checkpoint_path,
    device=device,
    sample_size=CONFIG['evaluation']['sample_size'],
    mc_paths=CONFIG['evaluation']['mc_paths'],
    seed=CONFIG['data']['seed'],
    fig_dir=FIG_DIR / 'oos',
    surface_grid=CONFIG['evaluation']['surface_grid'],
)
with open(RUN_RESULTS_DIR / 'oos_metrics.json', 'w', encoding='utf-8') as fp:
    json.dump(oos_metrics, fp, indent=2)
oos_metrics

## 8. Scorecard vs. Milestone Targets
The table below compares measured metrics against the success thresholds defined in the milestone
report. Out-of-distribution entries capture the degradation ratios relative to the validation split.

In [ ]:
# 8) Scorecard computation
from IPython.display import Markdown, display

success_targets = CONFIG.get('success_targets', {})
score_rows = []

def _format_label(metric_key: str) -> str:
    return metric_key.replace('_', ' ').title()

for metric, target in success_targets.items():
    if metric.startswith('ood_') and metric.endswith('_ratio'):
        base_key = metric[len('ood_'):-len('_ratio')]
        value = ood_metrics['ratios_vs_val'].get(base_key)
    elif metric == 'gamma_tv_ratio':
        value = val_metrics.get('gamma_tv_ratio')
    else:
        value = val_metrics.get(metric)

    if value is None or np.isnan(value):
        status = '⚪️'
        display_value = 'n/a'
    else:
        status = '✅' if value <= target else '⚠️'
        display_value = f"{value:.4g}"

    score_rows.append((_format_label(metric), display_value, f"{target:.4g}", status))

lines = ["| Metric | Value | Target | Status |", "|---|---|---|---|"]
for label, value, target, status in score_rows:
    lines.append(f"| {label} | {value} | {target} | {status} |")

display(Markdown("\n".join(lines)))

summary = {
    'validation': val_metrics,
    'ood': ood_metrics,
    'oos': oos_metrics,
}
summary


In [ ]:
# 8.1 Supplementary Artifacts
from IPython.display import display, Markdown, JSON, IFrame
import json

def _embed_html(title: str, path, height: int = 520):
    display(Markdown(f"**{title}**"))
    if path.exists():
        rel_path = path.relative_to(PROJECT_ROOT)
        display(IFrame(src=rel_path.as_posix(), width='100%', height=height))
    else:
        display(Markdown(f"_Missing: {path}_"))

# Training history snapshot
history_path = RUN_RESULTS_DIR / 'training_history.json'
display(Markdown('**Training History (last 3 epochs)**'))
if history_path.exists():
    with open(history_path) as fp:
        hist = json.load(fp)
    display(JSON(hist[-3:]))
else:
    display(Markdown('_No training history found._'))

# Loss curves
_embed_html('Loss Curves (linear scale)', FIG_DIR / 'loss_curves.html')
_embed_html('Loss Curves (log scale)', FIG_DIR / 'loss_curves_log.html')

# Out-of-sample dashboards
oos_dir = FIG_DIR / 'oos'
display(Markdown('**Out-of-Sample Dashboards**'))
if oos_dir.exists():
    html_files = sorted(oos_dir.glob('*.html'))
    if html_files:
        for html_path in html_files:
            title = html_path.stem.replace('_', ' ').title()
            _embed_html(title, html_path, height=420)
    else:
        display(Markdown('_No OOS HTML files found._'))
else:
    display(Markdown('_No OOS figures directory found._'))

# OOS metrics JSON
oos_metrics_path = RUN_RESULTS_DIR / 'oos_metrics.json'
display(Markdown('**Latest OOS Metrics**'))
if oos_metrics_path.exists():
    with open(oos_metrics_path) as fp:
        display(JSON(json.load(fp)))
else:
    display(Markdown('_OOS metrics JSON not found._'))


## 9. Findings
- Validation targets focus on price RMSE and Greek MAE; the accompanying scorecard cell reports the
  latest measurements alongside the milestone thresholds.
- Gamma smoothness is quantified via the total-variation ratio on a mid-maturity surface to ensure
  hedgeable curvature. Ratios above the target flag scenarios where the Sobolev penalty requires
  retuning.
- The volatility stress test provides the degradation ratios for $\sigma \in [0.60, 0.65]$, making it
  easy to verify the OOD criterion (
  $<10\%$ degradation) without re-running plots.
- Out-of-sample benchmarks remain available via `evaluate_oos`, including finite-difference and Monte
  Carlo baselines, so downstream reports can reference the same JSON/HTML artifacts listed above.
